In [ ]:
import torch

In [ ]:
torch.__version__

In [ ]:
# 자원관리를 야무지게 해보아요

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
pip install kagglehub[pandas-datasets]

In [ ]:
!pip install ipywidgets

!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension install --py widgetsnbextension

!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [ ]:
# Lane Detection dataset download -> TuSimple (21.6G)

import kagglehub

# Download latest version
path = kagglehub.dataset_download("manideep1108/tusimple")

print("Path to dataset files:", path)

In [ ]:
# img 슛

import kagglehub

# Download latest version
path = kagglehub.dataset_download("solesensei/solesensei_bdd100k")

print("Path to dataset files:", path)

In [ ]:
# 사용할 img dataset -> txt로

import os, json, shutil

# ─── 1) 경로 설정 ──────────────────────────────────────────────────────────
dataset_root = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2"
# train JSON 레이블
json_path    = os.path.join(
    dataset_root,
    "bdd100k_labels_release", "bdd100k", "labels",
    "bdd100k_labels_images_train.json"
)
# 실제 10k subset train 이미지 폴더 (FOUND 된 경로)
image_dir    = os.path.join(
    dataset_root,
    "bdd100k", "bdd100k", "images", "10k", "train"
)
# 결과물 저장 폴더
out_root     = os.path.join(dataset_root, "100k_extracted")
os.makedirs(out_root, exist_ok=True)

# ─── 2) 카테고리 판별 함수 ─────────────────────────────────────────────────
def categorize(entry):
    w = entry["attributes"]["weather"]
    t = entry["attributes"]["timeofday"]
    if w == "clear" and t == "daytime":
        return "clear_day"
    if t == "dawn/dusk":
        return "dawn_dusk"
    if w == "rainy" and t == "night":
        return "rainy_night"
    return None

# ─── 3) JSON 읽어서 해당 그룹별 파일명 수집 ────────────────────────────────
with open(json_path, "r") as f:
    anns = json.load(f)

groups = {"clear_day": [], "dawn_dusk": [], "rainy_night": []}
for e in anns:
    cat = categorize(e)
    if cat:
        groups[cat].append(e["name"])

# ─── 4) 그룹별 폴더에 이미지 복사 ───────────────────────────────────────────
for cat, names in groups.items():
    tgt = os.path.join(out_root, cat)
    os.makedirs(tgt, exist_ok=True)
    copied = 0

    for fn in names:
        src = os.path.join(image_dir, fn)
        if os.path.exists(src):
            shutil.copy(src, os.path.join(tgt, fn))
            copied += 1

    print(f"[{cat}] 전체 {len(names)}개 중 {copied}개 복사 → {tgt}")

# ─── 5) 그룹별 리스트 파일 저장(원한다면) ─────────────────────────────────
for cat in groups:
    lst = sorted(os.listdir(os.path.join(out_root, cat)))
    with open(f"{cat}_100k_list.txt", "w") as fw:
        fw.write("\n".join(lst))
    print(f"{cat}_100k_list.txt 생성 ({len(lst)}장)")

In [ ]:
# 노이즈 삽입 (glare, bloom, gamma)

from tqdm.notebook import tqdm
import os, random, csv
import numpy as np
from PIL import Image, ImageFilter

# ─── 0) 경로 설정 ───────────────────────────────────────────────────────
# dataset_root   = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2"
# extracted_root = os.path.join(dataset_root, "100k_extracted")   # clear_day, dawn_dusk, rainy_night 폴더
# aug_root       = os.path.join(dataset_root, "aug_8levels")
# os.makedirs(aug_root, exist_ok=True)

# ─── 1) 그룹별 리스트 파일 & 원본 폴더 매핑 ─────────────────────────────
# 현재 작업 디렉토리(노트북 파일이 있는 곳)
work_dir = os.getcwd()  

groups = {
    "clear_day"  : {
        "list": os.path.join(work_dir, "clear_day_100k_list.txt"),
        "src" : os.path.join(extracted_root, "clear_day")
    },
    "dawn_dusk"  : {
        "list": os.path.join(work_dir, "dawn_dusk_100k_list.txt"),
        "src" : os.path.join(extracted_root, "dawn_dusk")
    },
    "rainy_night": {
        "list": os.path.join(work_dir, "rainy_night_100k_list.txt"),
        "src" : os.path.join(extracted_root, "rainy_night")
    },
}

# ─── 2) 노이즈 레벨 정의 ─────────────────────────────────────────────────
noise_levels = list(range(1, 9))  # 1,2,...,8

# ─── 3) light noise 함수 ─────────────────────────────────────────────────
def add_light_noise_with_level(img: Image.Image, level: int) -> Image.Image:
    arr = np.array(img).astype(np.float32)

    # Bloom: blur radius & intensity ∝ level
    radius    = 5 * level
    intensity = 0.15 * level
    blur = img.filter(ImageFilter.GaussianBlur(radius))
    arr = np.clip(arr + intensity * np.array(blur), 0, 255)

    # Glare: random center, radius ∝ level
    h, w, _ = arr.shape
    cx = int(w * random.uniform(0.3, 0.7))
    cy = int(h * random.uniform(0.0, 0.3))
    r  = int(min(h, w) * 0.1 * level)
    y, x = np.ogrid[:h, :w]
    mask = np.exp(-((x-cx)**2 + (y-cy)**2) / (2*(r/2)**2))
    arr = np.clip(arr + (mask[...,None] * (0.1 * level)), 0, 255)

    # Gamma correction ∝ level
    gamma = 1.0 + 0.1 * level
    inv = 1.0 / gamma
    table = np.array([((i/255.0)**inv)*255 for i in range(256)]).astype(np.uint8)
    arr = table[arr.astype(np.uint8)]

    return Image.fromarray(arr.astype(np.uint8))

# ─── 4) CSV 메타파일 준비 ───────────────────────────────────────────────
meta_csv = os.path.join(aug_root, "noise_meta_8levels.csv")
with open(meta_csv, "w", newline="") as cf:
    writer = csv.writer(cf)
    writer.writerow(["group","original_image","aug_image","noise_level"])

    for grp, info in groups.items():
        with open(info["list"], 'r') as f:
            names = [ln.strip() for ln in f if ln.strip()]

        for lvl in noise_levels:
            pbar = tqdm(names,
                        desc=f"{grp} | level {lvl}",
                        unit="img",
                        mininterval=1.0)  # 출력 빈도 제한
            tgt_dir = os.path.join(aug_root, grp, f"noise_{lvl}")
            os.makedirs(tgt_dir, exist_ok=True)

            for fn in pbar:
                aug_name = f"{fn[:-4]}_lvl{lvl}.png"
                aug_path = os.path.join(tgt_dir, aug_name)

                # ── 이미 변환된 이미지면 건너뛰기 ──
                if os.path.exists(aug_path):
                    continue

                try:
                    img = Image.open(os.path.join(info["src"], fn)).convert("RGB")
                    aug = add_light_noise_with_level(img, level=lvl)
                    aug.save(aug_path)
                    writer.writerow([grp, fn, aug_name, lvl])
                except Exception as e:
                    print(f"[ERROR] {fn} | level {lvl} | {e}")
                    continue

            pbar.close()
        print(f"[{grp}] 완료")

print("=== 모든 증강 완료 ===")

In [2]:
!pip install matplotlib pillow tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 1.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 1.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# random image 보기
import os
import random
import matplotlib.pyplot as plt
from PIL import Image

dataset_root   = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2"
extracted_root = os.path.join(dataset_root, "100k_extracted")   # clear_day, dawn_dusk, rainy_night 폴더
aug_root       = os.path.join(dataset_root, "aug_8levels")
os.makedirs(aug_root, exist_ok=True)

def preview_random_images(group="dawn_dusk", level=3, num_images=5):
    folder = os.path.join(aug_root, group, f"noise_{level}")
    image_files = [f for f in os.listdir(folder) if f.endswith(".png")]
    chosen = random.sample(image_files, min(num_images, len(image_files)))

    plt.figure(figsize=(15, 3))
    for i, img_name in enumerate(chosen):
        img_path = os.path.join(folder, img_name)
        img = Image.open(img_path)

        plt.subplot(1, num_images, i+1)
        plt.imshow(img)
        plt.axis("off")
        plt.title(img_name)
    plt.tight_layout()
    plt.show()

# 예시 실행

num = random.randint(1, 8)  # 1 이상 8 이하 (양 끝 포함)
preview_random_images(group="rainy_night", level=num)

In [ ]:
# noise level에 따른 image 비교

import os
import matplotlib.pyplot as plt
from PIL import Image

def preview_all_levels(aug_root, group, base_name):
    plt.figure(figsize=(20, 4))
    
    for lvl in range(1, 9):
        folder = os.path.join(aug_root, group, f"noise_{lvl}")
        aug_name = f"{base_name}_lvl{lvl}.png"
        path = os.path.join(folder, aug_name)

        if not os.path.exists(path):
            print(f"❌ 파일 없음: {path}")
            continue

        img = Image.open(path)

        plt.subplot(1, 8, lvl)
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"Level {lvl}")
    
    plt.suptitle(f"Noise Level Comparison for '{base_name}'", fontsize=16)
    plt.tight_layout()
    plt.show()

aug_root = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/aug_8levels"
group = "rainy_night"              # 해당 이미지가 속한 그룹
base_name = "657d369a-e6f8727d"    # 확장자 및 "_lvlX" 제외한 부분

preview_all_levels(aug_root, group, base_name)

In [ ]:
# 원본 이미지 -> dataset에 포함

import os
import shutil

# 원본 이미지가 들어있는 3개 폴더
src_dirs = [
    "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/100k_extracted/clear_day",
    "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/100k_extracted/dawn_dusk",
    "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/100k_extracted/rainy_night"
]

# 복사할 대상 디렉토리 (img/)
dst_dir = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/aug_8levels/img"
os.makedirs(dst_dir, exist_ok=True)

count = 0
for src_dir in src_dirs:
    for fname in os.listdir(src_dir):
        if fname.endswith(".jpg"):
            base = os.path.splitext(fname)[0]  # 'abc123'
            new_name = f"{base}_lv0.jpg"
            src_path = os.path.join(src_dir, fname)
            dst_path = os.path.join(dst_dir, new_name)
            shutil.copy(src_path, dst_path)
            count += 1

print(f"✅ 총 {count}개의 .jpg 원본 이미지가 '_lv0.jpg'로 이름 변경되어 img 폴더에 복사 완료되었습니다.")

In [ ]:
# 그냥 기본 Resnet18로 학습, Overfitting 검증도 없는 그냥 .. 허허

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import random
from tqdm import tqdm

# ────────────────────────
# 1. Config & Hyperparams
# ────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/aug_8levels/img"  # <- 이미지 루트 디렉토리
batch_size = 128
embedding_dim = 128
num_epochs = 50
temperature = 0.5
use_resnet50 = True  # True 시 ResNet50 사용

# ────────────────────────
# 2. Dataset Class
# ────────────────────────
class ContrastiveImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.sample_groups = self._load_samples()

    def _load_samples(self):
        samples = {}
        for fname in os.listdir(self.root_dir):
            if not (fname.endswith(".png") or fname.endswith(".jpg")):
                continue
            if "_lv" not in fname:
                continue
            key = fname.split("_lv")[0]  # 'abc123_lv1.png' → 'abc123'
            samples.setdefault(key, []).append(fname)

        return list(samples.items())

    def __len__(self):
        return len(self.sample_groups)

    def __getitem__(self, idx):
        base_name, variants = self.sample_groups[idx]
        variants.sort()  # lv0 ~ lv8 순서 정렬 보장
        i = random.randint(0, len(variants)-2)
        j = i + 1  # 인접한 level끼리 positive pair 구성
        pos_files = [variants[i], variants[j]]
    
        img1 = Image.open(os.path.join(self.root_dir, pos_files[0])).convert('RGB')
        img2 = Image.open(os.path.join(self.root_dir, pos_files[1])).convert('RGB')
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2

# ────────────────────────
# 3. Model Backbone + Projection
# ────────────────────────
class SimCLRModel(nn.Module):
    def __init__(self, backbone='resnet18', projection_dim=128):
        super().__init__()
        base = torchvision.models.resnet50(weights=None) if backbone == 'resnet50' else torchvision.models.resnet18(weights=None)
        self.encoder = nn.Sequential(*list(base.children())[:-1])  # remove FC
        feat_dim = base.fc.in_features
        self.projector = nn.Sequential(
            nn.Linear(feat_dim, 512),
            nn.ReLU(),
            nn.Linear(512, projection_dim)
        )
    def forward(self, x):
        h = self.encoder(x).squeeze()
        z = self.projector(h)
        return F.normalize(z, dim=1)

# ────────────────────────
# 4. NT-Xent Loss
# ────────────────────────
def nt_xent_loss(z1, z2, temperature=0.5):
    N = z1.size(0)
    z = torch.cat([z1, z2], dim=0)
    sim = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=2)
    sim /= temperature

    labels = torch.arange(N).repeat(2)
    labels = (labels + N * (torch.arange(2).repeat_interleave(N))).to(z.device)
    mask = torch.eye(2*N, dtype=torch.bool).to(z.device)
    sim = sim.masked_fill(mask, float('-inf'))

    positives = torch.cat([torch.diag(sim, N), torch.diag(sim, -N)])
    numerator = torch.exp(positives)
    denominator = torch.exp(sim).sum(dim=1)
    loss = -torch.log(numerator / denominator).mean()
    return loss

# ────────────────────────
# 5. Training
# ────────────────────────
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # 범위 축소
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = ContrastiveImageDataset(data_dir, transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

model = SimCLRModel(backbone='resnet50' if use_resnet50 else 'resnet18', projection_dim=embedding_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for x1, x2 in tqdm(dataloader):
        x1, x2 = x1.to(device), x2.to(device)
        z1, z2 = model(x1), model(x2)
        loss = nt_xent_loss(z1, z2, temperature)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"[Epoch {epoch+1}] Loss: {avg_loss:.4f}")

In [3]:
# Resnet50으로 학습(Val set에 대해 검증 후, overfitting 조짐있으면 바로 early stopping)

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import random
from tqdm import tqdm

# ────────────────────────
# 1. Config & Hyperparams
# ────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/aug_8levels/img"
batch_size = 64
embedding_dim = 256
num_epochs = 50
temperature = 0.5
use_resnet50 = False
early_stop_patience = 10  # Early stopping 기준

# ────────────────────────
# 2. Dataset Class
# ────────────────────────
class ContrastiveImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.sample_groups = self._load_samples()

    def _load_samples(self):
        samples = {}
        for fname in os.listdir(self.root_dir):
            if not (fname.endswith(".png") or fname.endswith(".jpg")):
                continue
            if "_lv" not in fname:
                continue
            key = fname.split("_lv")[0]
            samples.setdefault(key, []).append(fname)
        return list(samples.items())

    def __len__(self):
        return len(self.sample_groups)

    def __getitem__(self, idx):
        base_name, variants = self.sample_groups[idx]
        variants.sort()
        i = random.randint(0, len(variants)-2)
        j = i + 1
        pos_files = [variants[i], variants[j]]

        img1 = Image.open(os.path.join(self.root_dir, pos_files[0])).convert('RGB')
        img2 = Image.open(os.path.join(self.root_dir, pos_files[1])).convert('RGB')
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2

# ────────────────────────
# 3. Model + Projection Head
# ────────────────────────
class SimCLRModel(nn.Module):
    def __init__(self, backbone='resnet18', projection_dim=128):
        super().__init__()
        base = torchvision.models.resnet50(weights=None) if backbone == 'resnet50' else torchvision.models.resnet18(weights=None)
        self.encoder = nn.Sequential(*list(base.children())[:-1])
        feat_dim = base.fc.in_features
        self.projector = nn.Sequential(
            nn.Linear(feat_dim, 512),
            nn.ReLU(),
            nn.Linear(512, projection_dim)
        )
    def forward(self, x):
        h = self.encoder(x).squeeze()
        z = self.projector(h)
        return F.normalize(z, dim=1)

# ────────────────────────
# 4. NT-Xent Loss
# ────────────────────────
def nt_xent_loss(z1, z2, temperature=0.5):
    N = z1.size(0)
    z = torch.cat([z1, z2], dim=0)
    sim = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=2)
    sim /= temperature

    labels = torch.arange(N).repeat(2)
    labels = (labels + N * (torch.arange(2).repeat_interleave(N))).to(z.device)
    mask = torch.eye(2*N, dtype=torch.bool).to(z.device)
    sim = sim.masked_fill(mask, float('-inf'))

    positives = torch.cat([torch.diag(sim, N), torch.diag(sim, -N)])
    numerator = torch.exp(positives)
    denominator = torch.exp(sim).sum(dim=1)
    loss = -torch.log(numerator / denominator).mean()
    return loss

# ────────────────────────
# 5. DataLoader & Transform
# ────────────────────────
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = ContrastiveImageDataset(data_dir, transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ────────────────────────
# 6. Training Loop + Early Stopping
# ────────────────────────
model = SimCLRModel(backbone='resnet50' if use_resnet50 else 'resnet18', projection_dim=embedding_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=4e-3)

best_val_loss = float('inf')
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for x1, x2 in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x1, x2 = x1.to(device), x2.to(device)
        z1, z2 = model(x1), model(x2)
        loss = nt_xent_loss(z1, z2, temperature)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x1, x2 in val_loader:
            x1, x2 = x1.to(device), x2.to(device)
            z1, z2 = model(x1), model(x2)
            loss = nt_xent_loss(z1, z2, temperature)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # Early Stopping Logic
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model2.pth")  # Save best model
    else:
        patience_counter += 1
        print(f"→ No improvement. Patience: {patience_counter}/{early_stop_patience}")

        if patience_counter >= early_stop_patience:
            print("⏹️ Early stopping triggered.")
            break

Epoch 1: 100%|██████████| 20/20 [00:21<00:00,  1.08s/it]


[Epoch 1] Train Loss: 4.0410 | Val Loss: 4.5720


Epoch 2: 100%|██████████| 20/20 [00:21<00:00,  1.09s/it]


[Epoch 2] Train Loss: 3.7057 | Val Loss: 4.0455


Epoch 3: 100%|██████████| 20/20 [00:21<00:00,  1.09s/it]


...

[Epoch 48] Train Loss: 3.0128 | Val Loss: 3.3593


Epoch 49: 100%|██████████| 20/20 [00:21<00:00,  1.09s/it]


[Epoch 49] Train Loss: 3.0230 | Val Loss: 3.4564
→ No improvement. Patience: 1/10


Epoch 50: 100%|██████████| 20/20 [00:21<00:00,  1.08s/it]


[Epoch 50] Train Loss: 3.0107 | Val Loss: 3.5003
→ No improvement. Patience: 2/10


In [ ]:
# Best_model.pth

import os, random, math, numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import transforms

# ├── 1. Config & Device
# │
project_name     = "simclr_noise_robust"
data_dir         = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/aug_8levels/img"
save_path        = "best_model.pth"
batch_size       = 256
accum_steps      = 2
proj_dim         = 256
temperature      = 0.4
num_epochs       = 100
base_lr          = 1e-3
warn_collapse    = 0.3
early_patience   = 5

# GPU 확인 및 AMP 조건 판단
use_amp          = torch.cuda.is_available() and torch.cuda.get_device_name().lower().find("6000") >= 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.multiprocessing.set_start_method("spawn", force=True)

# ├── 2. Dataset Class
# │
class ContrastiveImageDataset(Dataset):
    def __init__(self, root):
        self.root = root
        self.groups = self._scan()

    def _scan(self):
        groups = {}
        for f in os.listdir(self.root):
            if "_lv" not in f or not f.endswith((".png", ".jpg")):
                continue
            key = f.split("_lv")[0]
            groups.setdefault(key, []).append(f)
        return list(groups.items())

    def __len__(self):
        return len(self.groups)

    def __getitem__(self, idx):
        _, group = self.groups[idx]
        f1, f2 = random.sample(group, 2)
        img1 = Image.open(os.path.join(self.root, f1)).convert("RGB")
        img2 = Image.open(os.path.join(self.root, f2)).convert("RGB")
        tf = transforms.Compose([
            transforms.Resize((224, 224)),  # Only resizing for tensor conversion
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])
        return tf(img1), tf(img2)

# ├── 3. Model (ResNet-18 + BN-ReLU Projector)
# │
def resnet18_backbone():
    m = torchvision.models.resnet18(weights=None)
    return nn.Sequential(*list(m.children())[:-2], nn.AdaptiveAvgPool2d((1, 1)))

class SimCLR(nn.Module):
    def __init__(self, proj_dim=256):
        super().__init__()
        self.encoder = resnet18_backbone()
        self.projector = nn.Sequential(
            nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, proj_dim, bias=False), nn.BatchNorm1d(proj_dim, affine=False)
        )

    def forward(self, x):
        h = self.encoder(x).flatten(1)
        z = F.normalize(self.projector(h), dim=1)
        return z

# ├── 4. NT-Xent Loss (cosine similarity)
# │
def nt_xent(z1, z2, T=0.5):
    z1, z2 = F.normalize(z1, dim=1), F.normalize(z2, dim=1)
    N = z1.size(0)
    z = torch.cat([z1, z2], dim=0)
    sim = torch.matmul(z, z.T) / T
    sim.fill_diagonal_(-float('inf'))
    target = torch.arange(N, 2*N, device=z.device)
    loss = F.cross_entropy(sim[:N], target) + F.cross_entropy(sim[N:], target - N)
    return 0.5 * loss

# ├── 5. DataLoader
# │
dataset = ContrastiveImageDataset(data_dir)
train_len = int(0.8 * len(dataset))
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, pin_memory=True)

# ├── 6. Train Loop
# │
model = SimCLR(proj_dim).to(device)
opt = torch.optim.Adam(model.parameters(), lr=base_lr, weight_decay=1e-4)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=num_epochs)
scaler = torch.amp.GradScaler(enabled=use_amp)

best_val, patience = math.inf, 0
for epoch in range(1, num_epochs + 1):
    model.train(); epoch_loss, neg_cos = 0.0, 0.0
    for step, (x1, x2) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}")):
        x1, x2 = x1.to(device), x2.to(device)
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
            z1, z2 = model(x1), model(x2)
            loss = nt_xent(z1, z2, temperature) / accum_steps
        scaler.scale(loss).backward()

        with torch.no_grad():
            neg_cos += (z1 @ z2.T).fill_diagonal_(0).mean().item()

        if (step + 1) % accum_steps == 0:
            scaler.step(opt)
            scaler.update()
            opt.zero_grad()

        epoch_loss += loss.item() * accum_steps

    neg_cos /= len(train_loader)
    if neg_cos > warn_collapse:
        print(f"⚠️  Collapse Warning: neg_cos={neg_cos:.3f}")

    # Validation
    model.eval(); val_loss = 0.0
    with torch.no_grad():
        for x1, x2 in val_loader:
            x1, x2 = x1.to(device), x2.to(device)
            with torch.amp.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
                z1, z2 = model(x1), model(x2)
                val_loss += nt_xent(z1, z2, temperature).item()
    val_loss /= len(val_loader)

    print(f"[Epoch {epoch}] train={epoch_loss/len(train_loader):.4f}, val={val_loss:.4f}")

    if val_loss < best_val:
        best_val, patience = val_loss, 0
        torch.save(model.state_dict(), save_path)
        print("  ✔️  Best model saved")
    else:
        patience += 1
        if patience >= early_patience:
            print("⏹️  Early stopping")
            break
    sched.step()


In [ ]:
!pip install numpy pillow tqdm torch torchvision

In [1]:
import os, random, math, numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import transforms

# ├── 1. Config & Device
project_name     = "simclr_noise_robust"
data_dir         = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/aug_8levels/img"
save_path        = "SGDR_model_a10.pth"
batch_size       = 256
accum_steps      = 1
proj_dim         = 128
temperature      = 0.3
num_epochs       = 500
base_lr          = 1.5e-3
warn_collapse    = 0.3
early_patience   = 15

use_amp          = torch.cuda.is_available() and torch.cuda.get_device_name().lower().find("6000") >= 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.multiprocessing.set_start_method("spawn", force=True)

# ├── 2. Dataset Class
class ContrastiveImageDataset(Dataset):
    def __init__(self, root):
        self.root = root
        self.groups = self._scan()

    def _scan(self):
        groups = {}
        for f in os.listdir(self.root):
            if "_lv" not in f or not f.endswith((".png", ".jpg")):
                continue
            key = f.split("_lv")[0]
            groups.setdefault(key, []).append(f)
        return list(groups.items())

    def __len__(self):
        return len(self.groups)

    def __getitem__(self, idx):
        _, group = self.groups[idx]
        f1, f2 = random.sample(group, 2)
        img1 = Image.open(os.path.join(self.root, f1)).convert("RGB")
        img2 = Image.open(os.path.join(self.root, f2)).convert("RGB")
        tf = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.5] * 3, [0.5] * 3)
        ])
        return tf(img1), tf(img2)

# ├── 3. Model (ResNet-18 + Enhanced Projector)
def resnet18_backbone():
    m = torchvision.models.resnet18(weights=None)
    for param in m.parameters():  # ✅ 학습 가능하게 보장
        param.requires_grad = True
    return nn.Sequential(*list(m.children())[:-2], nn.AdaptiveAvgPool2d((1, 1)))

class SimCLR(nn.Module):
    def __init__(self, proj_dim=128):
        super().__init__()
        self.encoder = resnet18_backbone()
        self.projector = nn.Sequential(
            nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, 512, bias=False), nn.BatchNorm1d(512), nn.ReLU(inplace=True),
            nn.Linear(512, proj_dim, bias=False), nn.BatchNorm1d(proj_dim, affine=False)
        )

    def forward(self, x):
        h = self.encoder(x).flatten(1)
        z = F.normalize(self.projector(h), dim=1)
        return z

# ├── 4. NT-Xent Loss
def nt_xent(z1, z2, T=0.5):
    z1, z2 = F.normalize(z1, dim=1), F.normalize(z2, dim=1)
    N = z1.size(0)
    z = torch.cat([z1, z2], dim=0)
    sim = torch.matmul(z, z.T) / T
    sim.fill_diagonal_(-float('inf'))
    target = torch.arange(N, 2*N, device=z.device)
    loss = F.cross_entropy(sim[:N], target) + F.cross_entropy(sim[N:], target - N)
    return 0.5 * loss

# ├── 5. DataLoader

dataset = ContrastiveImageDataset(data_dir)
train_len = int(0.8 * len(dataset))
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])

# NUM_WORKERS = 4  # 또는 16

train_loader = DataLoader(
    train_ds, batch_size=batch_size, shuffle=True,
    pin_memory=True)

val_loader = DataLoader(
    val_ds, batch_size=batch_size, shuffle=False,
    pin_memory=True)


# ├── 6. Training Loop
# ├── 6. Training Loop
model = SimCLR(proj_dim).to(device)
opt = torch.optim.Adam(model.parameters(), lr=base_lr, weight_decay=1e-4)

# ✅ CosineAnnealingWarmRestarts: SGDR 방식
# T_0: 초기 주기(epoch 단위), T_mult: 이후 주기 배수 증가
sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, T_0=50, T_mult=2)

scaler = torch.amp.GradScaler(enabled=use_amp)

best_val, patience = math.inf, 0
for epoch in range(1, num_epochs + 1):
    model.train(); epoch_loss, neg_cos = 0.0, 0.0
    for step, (x1, x2) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}")):
        x1, x2 = x1.to(device), x2.to(device)
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
            z1, z2 = model(x1), model(x2)
            loss = nt_xent(z1, z2, temperature) / accum_steps
        scaler.scale(loss).backward()

        with torch.no_grad():
            neg_cos += (z1 @ z2.T).fill_diagonal_(0).mean().item()

        if (step + 1) % accum_steps == 0:
            scaler.step(opt)
            scaler.update()
            opt.zero_grad()

        epoch_loss += loss.item() * accum_steps

    neg_cos /= len(train_loader)
    if neg_cos > warn_collapse:
        print(f"⚠️  Collapse Warning: neg_cos={neg_cos:.3f}")

    model.eval(); val_loss = 0.0
    with torch.no_grad():
        for x1, x2 in val_loader:
            x1, x2 = x1.to(device), x2.to(device)
            with torch.amp.autocast(device_type="cuda", dtype=torch.float16, enabled=use_amp):
                z1, z2 = model(x1), model(x2)
                val_loss += nt_xent(z1, z2, temperature).item()
    val_loss /= len(val_loader)

    print(f"[Epoch {epoch}] train={epoch_loss/len(train_loader):.4f}, val={val_loss:.4f}")

    if val_loss < best_val:
        best_val, patience = val_loss, 0
        torch.save(model.state_dict(), "SGDR_model_a10.pth")
        print("  ✔️  Best model saved")
    else:
        patience += 1
        if patience >= early_patience:
            print("⏹️  Early stopping")
            break

    # ✅ SGDR 스케줄 업데이트 (epoch 단위)
    sched.step(epoch + 1)

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
# 학습 모델 불러서 model instance / weight loading

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

# ─────────────────────────────────────────
# ✅ SimCLRModel 정의 (ResNet18/50 + projector)
# ─────────────────────────────────────────
class SimCLRModel(nn.Module):
    def __init__(self, backbone='resnet18', projection_dim=256):
        super().__init__()
        base = torchvision.models.resnet50(weights=None) if backbone == 'resnet50' else torchvision.models.resnet18(weights=None)
        self.encoder = nn.Sequential(*list(base.children())[:-1])  # FC 제거
        feat_dim = base.fc.in_features
        self.projector = nn.Sequential(
            nn.Linear(feat_dim, 512),
            nn.ReLU(),
            nn.Linear(512, projection_dim)
        )

    def forward(self, x):
        h = self.encoder(x).squeeze()
        z = self.projector(h)
        return F.normalize(z, dim=1)

# ─────────────────────────────────────────
# ✅ 모델 인스턴스 생성 및 가중치 로드
# ─────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimCLRModel(backbone='resnet18', projection_dim=256).to(device)
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.eval()

In [4]:
# Cosine Similarity 구하기

from torch.nn.functional import cosine_similarity
from tqdm.notebook import tqdm

# Evaluation: cosine similarity between embeddings of positive pairs
def evaluate(model, dataloader):
    model.eval()
    total_sim = 0.0
    count = 0
    with torch.no_grad():
        for x1, x2 in tqdm(dataloader):  # (anchor, positive) pair
            x1, x2 = x1.cuda(), x2.cuda()
            z1, z2 = model(x1), model(x2)
            sim = cosine_similarity(z1, z2).mean().item()
            total_sim += sim
            count += 1
    print(f"Average Cosine Similarity: {total_sim / count:.4f}")

evaluate("best_model2.pth", val_loader)

AttributeError: 'str' object has no attribute 'eval'

In [ ]:
import os
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# ✅ SimCLR 모델 정의 (encoder만 사용)
class EncoderOnly(torch.nn.Module):
    def __init__(self):
        super().__init__()
        resnet = torch.hub.load("pytorch/vision:v0.10.0", "resnet50", pretrained=False)
        self.encoder = torch.nn.Sequential(*list(resnet.children())[:-2], torch.nn.AdaptiveAvgPool2d((1, 1)))

    def forward(self, x):
        return self.encoder(x).squeeze(-1).squeeze(-1)

# ✅ 모델 로드
model = EncoderOnly()
ckpt = torch.load("best_model.pth", map_location="cpu")
state_dict = {k.replace("encoder.", ""): v for k, v in ckpt.items() if "encoder." in k}
model.load_state_dict(state_dict, strict=False)
model.eval()

# ✅ 이미지 경로 및 전처리 정의
base_path = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/aug_8levels/img"
prefix = "b19efc15-9583cded"
levels = [0, 3, 5, 8]

transform = transforms.Compose([
    transforms.Resize((360, 640)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ✅ feature 추출
features = {}
# 경로 지정 시 lv0은 jpg 확장자로 시도
for lv in levels:
    if lv == 0:
        path = os.path.join(base_path, f"{prefix}_lv{lv}.jpg")  # 예외처리: .jpg
    else:
        path = os.path.join(base_path, f"{prefix}_lv{lv}.png")

    if not os.path.exists(path):
        print(f"❌ Missing: {path}")
        continue

    img = Image.open(path).convert("RGB")
    x = transform(img).unsqueeze(0)
    with torch.no_grad():
        f = model(x).squeeze()
        features[lv] = f / f.norm()

# ✅ cosine similarity 계산
print("✅ Cosine Similarity with lv0 as anchor")
for lv in levels[1:]:
    sim = F.cosine_similarity(features[0], features[lv], dim=0).item()
    print(f"lv0 vs lv{lv}: {sim:.4f}")

# ✅ 시각화 (선택)
fig, axes = plt.subplots(1, 4, figsize=(12, 3))
for i, lv in enumerate([0, 3, 5, 8]):
    ext = "jpg" if lv == 0 else "png"
    path = os.path.join(base_path, f"{prefix}_lv{lv}.{ext}")
    print(f"🔎 불러오는 이미지 경로: {path}")
    
    img = Image.open(path).convert("RGB").copy()
    img = transforms.Resize((360, 640))(img)
    
    axes[i].imshow(img)
    axes[i].axis("off")
    axes[i].set_title(f"lv{lv}")
plt.tight_layout()
plt.show()

In [ ]:
import os
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision import transforms
from tqdm import tqdm

model = model.eval()
model.cuda()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

base_path = "/home1/hyunje0/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/aug_8levels/img"

file_dict = {}
for fname in os.listdir(base_path):
    if "_lv" in fname:
        prefix = fname.split("_lv")[0]
        lv = int(fname.split("_lv")[1].split(".")[0])
        file_dict.setdefault(prefix, {})[lv] = fname

valid_prefixes = [p for p, v in file_dict.items() if all(lv in v for lv in range(9))]

similarities = {f"lv0_vs_lv{lv}": [] for lv in range(1, 9)}

for prefix in tqdm(valid_prefixes[:1000]):  # 메모리 문제 예방
    feats = {}
    for lv in range(9):
        fname = file_dict[prefix][lv]
        path = os.path.join(base_path, fname)
        if not os.path.exists(path):
            continue
        img = Image.open(path).convert("RGB")
        x = transform(img).unsqueeze(0)
        feat = model(x.cuda()).squeeze().detach().cpu()
        feat = feat / feat.norm()
        feats[lv] = feat
    if 0 in feats:
        for lv in range(1, 9):
            if lv in feats:
                sim = F.cosine_similarity(feats[0], feats[lv], dim=0).item()
                similarities[f"lv0_vs_lv{lv}"].append(sim)


avg_sims = {k: f"{sum(v)/len(v):.4f}" for k, v in similarities.items()}
print(avg_sims)

In [7]:
!pip install torchsummary

In [8]:
import torch
from torchsummary import summary   # pip install torchsummary
model = torch.load('Models/best_model.pth', map_location='cpu')

# Encoder 부분만 보고 싶다면
encoder = model.encoder if hasattr(model, 'encoder') else model
print(encoder)              # 레이어 블록 이름·순서 출력
summary(encoder, (3, 224, 224))   # 출력 feature map 크기 확인

OrderedDict({'encoder.0.weight': tensor([[[[ 1.5123e-02,  3.7613e-02,  3.6524e-02,  ..., -1.3588e-02,
            6.6022e-03,  1.9597e-03],
          [ 6.8758e-03,  3.0995e-02, -1.4367e-02,  ...,  6.5958e-02,
           -8.9356e-05, -3.5220e-02],
          [-2.8409e-02,  2.8499e-02,  5.7666e-03,  ...,  1.7665e-02,
            1.3581e-02,  7.8884e-03],
          ...,
          [-7.9284e-03,  3.4920e-03,  6.6924e-03,  ...,  4.4625e-02,
            2.7215e-02, -2.0571e-02],
          [ 1.5916e-02, -2.6658e-02, -1.8799e-02,  ..., -1.9751e-02,
            2.2104e-02, -1.3472e-02],
          [ 3.8837e-02, -1.2203e-02,  2.5661e-03,  ...,  6.9068e-02,
           -1.7559e-02,  5.7015e-02]],

         [[-1.0309e-03, -6.5484e-03,  2.7907e-02,  ...,  2.3875e-02,
            8.3578e-03,  6.1388e-02],
          [ 3.7632e-02,  1.7614e-03,  1.7783e-02,  ...,  6.4293e-03,
            4.1810e-03,  3.1678e-02],
          [-2.8718e-02, -3.1290e-02, -5.5545e-03,  ..., -8.2018e-03,
           -1.1185e-02, -

AttributeError: 'collections.OrderedDict' object has no attribute 'apply'